# Coupled Model: Damage/Flow
We develop a coupled damage model, which has options to couple damage to flow, depending on user input. We apply this to the MISMIP+ geometry.

In [ ]:
# import everything
import firedrake
from firedrake import exp, conditional, max_value, sqrt, sym, grad, tr, det, interpolate, Constant, sin, cos, dx, inner, as_vector, tanh
import icepack
import matplotlib.pyplot as plt
import icepack.plot
import numpy as np
import cmocean
from icepack.constants import (
    ice_density as rho_i,
    water_density as rho_w,
    gravity as g,
    weertman_sliding_law as m,
    glen_flow_law as n,
)
from icepack.models.hybrid import horizontal_strain_rate, vertical_strain_rate, stresses
import tqdm
from operator import itemgetter

## Define a bunch of functions

### Geometry Functions

In [ ]:
def mismip_bed_2d(mesh):

    """
    This function defines the bed geometry based on MISMIP+
    
    Inputs: 
    ----------
    mesh: 2D mesh 
    
    Outputs: 
    ----------
    bed elevation
    
    """
    x, y = firedrake.SpatialCoordinate(mesh)

    x_c = Constant(300e3)
    X = x / x_c

    B_0 = Constant(-150)
    B_2 = Constant(-728.8)
    B_4 = Constant(343.91)
    B_6 = Constant(-50.57)
    B_x = B_0 + B_2 * X**2 + B_4 * X**4 + B_6 * X**6

    f_c = Constant(4e3)
    d_c = Constant(500)
    w_c = Constant(24e3)

    B_y = d_c * (
        1 / (1 + exp(-2 * (y - Ly / 2 - w_c) / f_c)) +
        1 / (1 + exp(+2 * (y - Ly / 2 + w_c) / f_c))
    )

    z_deep = Constant(-720)
    
    return max_value(B_x + B_y, z_deep)
    
def mismip_bed_3d(mesh):

    """
    This function defines the bed geometry based on MISMIP+
    
    Inputs: 
    ----------
    mesh: 2D mesh 
    
    Outputs: 
    ----------
    bed elevation
    
    """
    x, y,ζ = firedrake.SpatialCoordinate(mesh)

    x_c = Constant(300e3)
    X = x / x_c

    B_0 = Constant(-150)
    B_2 = Constant(-728.8)
    B_4 = Constant(343.91)
    B_6 = Constant(-50.57)
    B_x = B_0 + B_2 * X**2 + B_4 * X**4 + B_6 * X**6

    f_c = Constant(4e3)
    d_c = Constant(500)
    w_c = Constant(24e3)

    B_y = d_c * (
        1 / (1 + exp(-2 * (y - Ly / 2 - w_c) / f_c)) +
        1 / (1 + exp(+2 * (y - Ly / 2 + w_c) / f_c))
    )

    z_deep = Constant(-720)
    
    return max_value(B_x + B_y, z_deep)

### Model Parameterization Functions

In [ ]:
def friction(**kwargs):
    """
    This function defines a Weertman-style sliding law
    
    Inputs: 
    ----------
    velocity, thickness, surface elevation, friction coefficient C 
    
    Outputs: 
    ----------
    basal stress
    
    """
    variables = ('velocity', 'thickness', 'surface', 'friction')
    u, h, s, C = map(kwargs.get, variables)

    p_W = rho_w * g * max_value(0, -(s - h))
    p_I = rho_i * g * h
    N = max_value(0, p_I - p_W)
    τ_c = N / 2

    u_c = (τ_c / C)**m
    u_b = sqrt(inner(u, u))

    return τ_c * (
        (u_c**(1/m + 1) + u_b**(1/m + 1))**(m / (m + 1)) - u_c
    )

from icepack.models.viscosity import viscosity_depth_averaged
def viscosity_damaged(**kwargs):
    u = kwargs["velocity"]
    h = kwargs["thickness"]
    A = kwargs["fluidity"]
    D = kwargs["damage"]
    return viscosity_depth_averaged(
        velocity=u,
        thickness=h,
        fluidity=(1 - D)**(-n) * A,
    )

### Damage Model Functions

In [ ]:
from icepack.models.diagnostic_damage_transport import calcAdvectionDiagnosticDamage, calcStressThreshold

### Other Functions

In [ ]:
from icepack.mask_smooth_functions import flotationHeight, flotationMask, firedrakeSmooth
def mismip_meltfcn(omega,surface,thickness,bed):
    
    """
    This function calculates the basal melting rate by the parameterization in Asay-Davis et al. 2016.
    
    Inputs: 
    ----------
    omega: parameter in melting parameterization (1/yr)
    surface: 2D surface elevation field (m)
    thickness: 2D thickness field (m)
    bed: 2D bed elevation field (m)
    
    Outputs: 
    ----------
    -mdot: melt rate (negative to feed into SMB)
    """
    
    Q = thickness.function_space()

    zF = flotationHeight(bed,Q)
    floating,grounded = flotationMask(surface,zF,Q)
    zd = ((surface-thickness))*floating
    Hc = (zd-bed)*floating
    
    Hc0 = 75
    z0 = -100
    
    mdot = omega*tanh(Hc/Hc0)*max_value(z0-zd,0)
    
    return -mdot

### Run Function

In [ ]:
def run_simulation(couplings,solver,final_time,dt,bed,u,s,h,Aglen2,C,omega,smb,num_layers_D,D2_initial,σ_t,criterion,Dmax):
    
    """
    This function evolves flow with various couplings, including: damage, temperature, grain size. This includes an option for basal melting.
    
    Inputs: 
    ----------
    couplings: a dictionary of coupling options, user-defined
    solver: flow solver
    final_time: total amount of time to run the simulation for (yrs)
    dt: timestep (yrs)
    bed: bed elevation (2D) (m)
    u: 2D initial velocity (m/yr)
    s: 2D initial surface elevation (m)
    h: 2D initial thickness (m)
    Aglen2: fluidity on 2d mesh
    C: friction coefficient
    omega: scaling for basal melt rate
    smb: surface accumulation rate (m/yr)

    Parameters for Damage Model:
        num_layers_D: number of z layers for damage model
        D2_initial: initial (2D) damage field
        σ_t: threshold 
        criterion: the stress criterion for damage (options = vms for von Mises 
            stress, mps for maximum principal stress, hayhurst for the Hayhurst stress)
    Dmax: maximum value for damage fields
            
    Outputs: 
    ----------
    h_save: an array of thicknesses at each timestep
    u_save: an array of velocities at each timestep
    s_save: an array of elevations at each timestep
    gs_save: an array of (2D) grain sizes at each timestep
    D_save: an array of (3D) damage fields at each timestep
    E_save: an array of (3D) energy density fields at each timestep
    """

    num_steps = int(final_time / dt)
    progress_bar = tqdm.trange(num_steps)

    h_0 = h

    Q2 = h.function_space()
    V2 = u.function_space()

    D2 = D2_initial.copy(deepcopy=True)

    D_save = []
    u_save = []
    s_save = []
    h_save = []

    for step in progress_bar:

        melt = mismip_meltfcn(omega,s,h,bed)
        a = interpolate(melt+smb, Q2)

        """ calculate thickness and velocity """
        h = solver.prognostic_solve(
            dt,
            thickness=h,
            velocity=u,
            accumulation=a,
            thickness_inflow=h_0
        )
        s = icepack.compute_surface(thickness=h, bed=bed)

        u = solver.diagnostic_solve(
            velocity=u,
            thickness=h,
            surface=s,
            fluidity=Aglen2,
            friction=C,
            damage=D2,
        )


        """ calculate damage as a function of sigma_t(d), temperature """
        if couplings['stress_to_damage']:
            D2 = calcAdvectionDiagnosticDamage(
                dt = dt,
                velocity = u,
                thickness = h,
                surface = s,
                fluidity = Aglen2,
                num_layers = num_layers_D,
                criterion = criterion,
                threshold = σ_t,
                Dold = D2,
                Dmax = Dmax
            )

        """ save fields """
        h_save.append(h)
        u_save.append(u)
        s_save.append(s)
        D_save.append(D2)

        min_h = h.dat.data_ro.min()
        avg_h = firedrake.assemble(h * dx) / area
        description = f"avg, min h: {avg_h:4.2f}, {min_h:4.2f}"
        progress_bar.set_description(description)

    return h_save, u_save, s_save, D_save

## Set up Model geometry

Input the starting point and define the basic geometry.

In [ ]:
# set up model geometry
Lx, Ly = 640e3, 80e3
ny = 10
nx = int(Lx/Ly)*ny
area = Lx*Ly

with firedrake.CheckpointFile("231114_MISMIP+Geometry_SteadyState_MeshRefinementMethod_4500yr_1refinement.h5",'r') as checkpoint:
    mesh2 = checkpoint.load_mesh('firedrake_default')
    u_ss = checkpoint.load_function(mesh2,"velocity")
    h_ss = checkpoint.load_function(mesh2,"thickness")
    s_ss = checkpoint.load_function(mesh2,"surface")

Define function spaces and bed on 2D mesh

In [ ]:
Q2 = firedrake.FunctionSpace(mesh2, family='CG', degree=2)
V2 = firedrake.VectorFunctionSpace(mesh2, family='CG', degree=2)
bed2 = interpolate(mismip_bed_2d(mesh2), Q2)

Extrude the mesh for the damage solver.

In [ ]:
num_layers_D = 20
mesh3D = firedrake.ExtrudedMesh(mesh2, layers=num_layers_D)

Q3D = firedrake.FunctionSpace(mesh3D, family='CG', degree=2, vfamily='DG', vdegree=0)
V3D = firedrake.VectorFunctionSpace(mesh3D, dim=2, family='CG', degree=2, vfamily='GL', vdegree=0)

bed3D = interpolate(mismip_bed_3d(mesh3D), Q3D)

## Define the model

In [ ]:
# define flow model with damage
flow_model = icepack.models.IceStream(friction=friction,viscosity=viscosity_damaged)
opts = {
    'dirichlet_ids': [1],
    'side_wall_ids': [3, 4],
    'diagnostic_solver_type': 'petsc',
    'diagnostic_solver_parameters': {
        'snes_type': 'newtontr',
        'ksp_type': 'preonly',
        'pc_type': 'lu',
        'pc_factor_mat_solver_type': 'mumps'
    }
}
flow_solver = icepack.solvers.FlowSolver(flow_model, **opts)

## Initialize the model

Basal Friction

In [ ]:
C = firedrake.Constant(0.01)

Temperature

In [ ]:
meltfrac = Constant(0.0)
T = Constant(260)
Aglen2 = interpolate(icepack.rate_factor(T),Q2)

Damage

In [ ]:
D2_initial = firedrake.Function(Q2)

## Now Run Model

In [ ]:
# simulation parameters
final_time = 10
dt = 0.25
smb = Constant(0.3)
omega = 0.2

In [ ]:
# Define couplings
couplings = {'stress_to_damage': True} # damage evolution

In [ ]:
# user input parameters
σ_t = interpolate(Constant(0.1),Q2) # stress threshold.
criterion = 'hayhurst' # options = hayhurst, vms, mps
Dmax = firedrake.Constant(0.79)

In [ ]:
h_save, u_save, s_save, D_save = run_simulation(
    couplings,flow_solver,final_time,dt,bed2,
    u_ss,s_ss,h_ss,Aglen2,C,
    omega,smb,
    num_layers_D,D2_initial,σ_t,criterion,Dmax
)

In [ ]:
fig, axes = icepack.plot.subplots()
colors = firedrake.tripcolor(D_save[-1], axes=axes)
CS=plt.colorbar(colors, cmap="viridis", ax=axes, orientation='horizontal')
CS.set_label(r'Damage',fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = icepack.plot.subplots()
colors = firedrake.tripcolor(firedrake.interpolate(u_save[-1],V2), axes=axes)
CS=plt.colorbar(colors, cmap="viridis", ax=axes, orientation='horizontal')
CS.set_label(r'Velocity (m/yr)',fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = icepack.plot.subplots()
colors = firedrake.tripcolor(firedrake.interpolate(u_save[-1]-u_save[0],V2), axes=axes)
CS=plt.colorbar(colors, cmap="viridis", ax=axes, orientation='horizontal')
CS.set_label(r'Velocity Change (m/yr)',fontsize=12)
plt.tight_layout()
plt.show()

## Save Fields

In [ ]:
us = firedrake.Function(V2, name="us")
hs = firedrake.Function(Q2, name="hs")
ss = firedrake.Function(Q2, name="ss")
Ds = firedrake.Function(Q3D, name="Ds")

num_steps = int(final_time / dt)
ts = tqdm.trange(num_steps)
with firedrake.CheckpointFile("CoupledDamageModel_Geometry=MISMIP+_tf=100yr_dt=0.25_omega=0.2_couplings=stress2damage_Tinitial=250K_Dinitial=0_Dmax=0.79_criterion=hayhurst_σt=0.1MPa.h5", 'w') as checkpoint:
    checkpoint.save_mesh(mesh2d)  
    for i in ts:
        us.interpolate(u_save[i])
        checkpoint.save_function(us, idx=i)

        hs.interpolate(h_save[i])
        checkpoint.save_function(hs, idx=i)

        ss.interpolate(s_save[i])
        checkpoint.save_function(ss, idx=i)
        
        Ds.interpolate(D_save[i])
        checkpoint.save_function(Ds, idx=i)

